# Imports

In [1]:
config = {
    'dhdt': {
        'depth': 3,
        'learning_rate': 0.01,#1e-3,
        
        'initializer': 'GlorotUniform', #GlorotUniform
        
        'loss': 'binary_crossentropy',#'mae',
        'optimizer': 'adam',        
        
        'beta_1': 10,
        'beta_2': 50,
        
        'squeeze_factor': 1,
        
        'batch_size': 512,
        'epochs': 1_000,
        'early_stopping_epochs': 50,
    },
    
    
    
    'make_classification': {
        'number_of_variables': 5,
        'n_samples': 5_000,
        'num_eval': 10,
    },

    'computation': {
        'random_seed': 42,
        'trials': 5,
        'n_jobs': 50,
        'verbosity': 0,
    },
}



In [2]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '' #'true'

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities import *
from utilities.DHDT import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy

# Evaluation

## make_classification

In [3]:
if False:
    config_test = deepcopy(config)
    config_test['make_classification']['n_samples'] = 10_000
    config_test['dhdt']['epochs'] = 3

    dataset_dict = {}
    model_dict = {}

    scores_dict = {'sklearn': {},
                   'DHDT': {}}

    dataset_dict = get_preprocessed_dataset('make_classification',
                                            random_seed=config_test['computation']['random_seed'],
                                            config=config_test['make_classification'],
                                            verbosity=1)

    model_dict['sklearn'] = DecisionTreeClassifier(max_depth=3, 
                                                   random_state=config_test['computation']['random_seed'])

    model_dict['sklearn'].fit(dataset_dict['X_train'], 
                              dataset_dict['y_train'])



    model_dict['DHDT'] = DHDT(dataset_dict['X_train'].shape[1],

                                depth = config_test['dhdt']['depth'],

                                learning_rate = config_test['dhdt']['learning_rate'],
                                optimizer = config_test['dhdt']['optimizer'],

                                beta_1 = config_test['dhdt']['beta_1'],
                                beta_2 = config_test['dhdt']['beta_2'],

                                squeeze_factor = config_test['dhdt']['squeeze_factor'],

                                loss = config_test['dhdt']['loss'],#'mae',

                                random_seed = config_test['computation']['random_seed'],
                                verbosity = 2)        


    scores_dict['history'] = model_dict['DHDT'].fit(dataset_dict['X_train'], 
                                                  dataset_dict['y_train'], 
                                                  batch_size=config_test['dhdt']['batch_size'], 
                                                  epochs=config_test['dhdt']['epochs'], 
                                                  early_stopping_epochs=config_test['dhdt']['early_stopping_epochs'], 
                                                  valid_data=(dataset_dict['X_valid'], dataset_dict['y_valid']))



    scores_dict['sklearn']['accuracy_test'] = model_dict['sklearn'].score(dataset_dict['X_test'], 
                                                                     dataset_dict['y_test'])


    dataset_dict['y_test_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_test'])
    scores_dict['DHDT']['accuracy_test'] = accuracy_score(dataset_dict['y_test'], np.round(dataset_dict['y_test_dhdt']))


In [4]:
if True:
    parallel_eval_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_synthetic = parallel_eval_synthetic(delayed(evaluate_synthetic_parallel)(index = index,
                                                                                                random_seed_data = config['computation']['random_seed']+index,
                                                                                                random_seed_model = config['computation']['random_seed'],#+random_seed_model,
                                                                                                config = config,
                                                                                                verbosity = -1) for index in range(config['make_classification']['num_eval']))

    for i, synthetic_result in enumerate(evaluation_results_synthetic):
        if i == 0:
            model_dict_synthetic = synthetic_result[0]
            scores_dict_synthetic = synthetic_result[1]
            dataset_dict_synthetic = synthetic_result[2]
        else: 
            model_dict_synthetic = mergeDict(model_dict_synthetic, synthetic_result[0])
            scores_dict_synthetic = mergeDict(scores_dict_synthetic, synthetic_result[1])
            dataset_dict_synthetic = mergeDict(dataset_dict_synthetic, synthetic_result[2])


    metrics = ['accuracy_test']
    index = [i for i in range(config['make_classification']['num_eval'])]
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_synthetic[i]['DHDT'][metrics[0]] for i in range(config['make_classification']['num_eval'])]

    scores_sklearn = [scores_dict_synthetic[i]['sklearn'][metrics[0]] for i in range(config['make_classification']['num_eval'])]


    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['make_classification']['num_eval'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['make_classification']['num_eval'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_synthetic = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_synthetic)
    display(scores_dataframe_synthetic[scores_dataframe_synthetic.columns[1::3]])
    display(scores_dataframe_synthetic.describe())

[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   3 out of  10 | elapsed:  4.3min remaining: 10.1min
[Parallel(n_jobs=50)]: Done   7 out of  10 | elapsed:  6.1min remaining:  2.6min
[Parallel(n_jobs=50)]: Done  10 out of  10 | elapsed:  7.0min finished


,DHDT accuracy_test_mean,DHDT accuracy_test_max,DHDT accuracy_test_std,sklearn accuracy_test_mean,sklearn accuracy_test_max,sklearn accuracy_test_std
0,0.7142,0.806,0.046348,0.7396,0.862,0.061207
1,0.8566,0.926,0.104280,0.9060,0.933,0.054000
2,0.8726,0.887,0.008452,0.9050,0.937,0.016000
3,0.7192,0.799,0.063203,0.8142,0.911,0.048400
4,0.6868,0.761,0.049648,0.7774,0.790,0.025200
5,0.7896,0.830,0.021134,0.8318,0.859,0.013600
6,0.7866,0.891,0.157152,0.8784,0.923,0.089200
7,0.6756,0.730,0.045715,0.7840,0.812,0.014000
8,0.7560,0.895,0.069811,0.8754,0.889,0.006800
9,0.6528,0.720,0.047943,0.6910,0.691,0.000000


,DHDT accuracy_test_max,sklearn accuracy_test_max
0,0.806,0.862
1,0.926,0.933
2,0.887,0.937
3,0.799,0.911
4,0.761,0.790
5,0.830,0.859
6,0.891,0.923
7,0.730,0.812
8,0.895,0.889
9,0.720,0.691


,DHDT accuracy_test_mean,DHDT accuracy_test_max,DHDT accuracy_test_std,sklearn accuracy_test_mean,sklearn accuracy_test_max,sklearn accuracy_test_std
count,10.000000,10.000000,10.000000,10.00000,10.00000,10.000000
mean,0.751000,0.824500,0.061369,0.82028,0.86070,0.032841
std,0.074934,0.073414,0.042554,0.07264,0.07777,0.028857
min,0.652800,0.720000,0.008452,0.69100,0.69100,0.000000
25%,0.693650,0.770500,0.045873,0.77905,0.82375,0.013700
50%,0.737600,0.818000,0.048796,0.82300,0.87550,0.020600
75%,0.788850,0.890000,0.068159,0.87765,0.92000,0.052600
max,0.872600,0.926000,0.157152,0.90600,0.93700,0.089200


## Real-World Eval

In [5]:
if True:

    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    #identifier_list = ['Habermans Survival']

    parallel_eval_real_world = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_real_world = parallel_eval_real_world(delayed(evaluate_real_world_parallel)(identifier_list=identifier_list, 
                                                                                                   random_seed_model=config['computation']['random_seed']+i,
                                                                                                   config = config,
                                                                                                   verbosity = -1) for i in range(config['computation']['trials']))


    for i, real_world_result in enumerate(evaluation_results_real_world):
        if i == 0:
            model_dict_real_world = real_world_result[0]
            scores_dict_real_world = real_world_result[1]
            dataset_dict_real_world = real_world_result[2]
        else: 
            model_dict_real_world = mergeDict(model_dict_real_world, real_world_result[0])
            scores_dict_real_world = mergeDict(scores_dict_real_world, real_world_result[1])
            dataset_dict_real_world = mergeDict(dataset_dict_real_world, real_world_result[2])

    metrics = ['accuracy_test']
    index = identifier_list
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_real_world[identifier]['DHDT'][metrics[0]] for identifier in identifier_list]

    scores_sklearn = [scores_dict_real_world[identifier]['sklearn'][metrics[0]] for identifier in identifier_list]

    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_real_world = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_real_world)
    display(scores_dataframe_real_world[scores_dataframe_real_world.columns[1::3]])



[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of   5 | elapsed: 24.1min remaining: 36.2min
[Parallel(n_jobs=50)]: Done   5 out of   5 | elapsed: 34.5min finished


,DHDT accuracy_test_mean,DHDT accuracy_test_max,DHDT accuracy_test_std,sklearn accuracy_test_mean,sklearn accuracy_test_max,sklearn accuracy_test_std
Adult,0.535657,0.767506,0.189751,0.815571,0.815571,0.000000e+00
Bank Marketing,0.470471,0.717540,0.214576,0.784672,0.784672,0.000000e+00
Loan Credit,0.482335,0.771779,0.177655,0.712556,0.712556,0.000000e+00
Credit Card,0.472167,0.782333,0.219264,0.776167,0.776167,0.000000e+00
Car,0.695652,0.776812,0.047699,0.898551,0.898551,1.110223e-16
Absenteeism,0.631081,0.648649,0.010978,0.635135,0.635135,0.000000e+00
Loan House,0.660656,0.786885,0.065697,0.762295,0.762295,0.000000e+00
Cervical Cancer,0.387135,0.935673,0.346886,0.467836,0.467836,0.000000e+00
Heart Disease,0.793333,0.816667,0.013333,0.850000,0.850000,0.000000e+00
Titanic,0.671910,0.780899,0.089578,0.797753,0.797753,0.000000e+00


,DHDT accuracy_test_max,sklearn accuracy_test_max
Adult,0.767506,0.815571
Bank Marketing,0.717540,0.784672
Loan Credit,0.771779,0.712556
Credit Card,0.782333,0.776167
Car,0.776812,0.898551
Absenteeism,0.648649,0.635135
Loan House,0.786885,0.762295
Cervical Cancer,0.935673,0.467836
Heart Disease,0.816667,0.850000
Titanic,0.780899,0.797753


In [6]:
if False:
    identifier = identifier_list[0]#"Absenteeism"
    plt.figure(figsize=(15,8))
    image = model_dict_real_world[identifier]['DHDT'].plot(normalizer_list=dataset_dict_real_world[identifier]['normalizer_list'])
    display(image)

    plt.figure(figsize=(15,8))
    plot_tree(model_dict_real_world[identifier]['sklearn'], fontsize=10) 
    plt.show()

# Hyperparameter Optimization

In [7]:
parameter_dict = {
        'depth': [3],
        'learning_rate': [0.05, 0.01, 0.005, 0.001, 0.0005],#[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
        
        'loss': ['binary_crossentropy'],#['binary_crossentropy', 'rmse'],#'mae',
        'optimizer': ['adam', 'sgd'],#['adam', 'sgd'],        
        
        'beta_1': [10, 50],#[10, 50, 100],
        'beta_2': [10, 50],#[10, 50, 100],
        
        'squeeze_factor': [0.2, 0.5, 1, 2, 5],#[0.2, 0.5, 1, 2, 5],    
}

parameter_dict = {
        #'depth': [3],
        #'learning_rate': [0.05, 0.01, 0.005, 0.001, 0.0005],#[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
        
        'loss': ['binary_crossentropy'],#['binary_crossentropy', 'rmse'],#'mae',
        'optimizer': ['adam', 'sgd'],#['adam', 'sgd'],        
        
        #'beta_1': [10, 50],#[10, 50, 100],
        #'beta_2': [10, 50],#[10, 50, 100],
        
        #'squeeze_factor': [0.2, 0.5, 1, 2, 5],#[0.2, 0.5, 1, 2, 5],    
}

parameter_grid = ParameterGrid(parameter_dict)

In [8]:
if False:
    parallel_hpo_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_hpo_synthetic = parallel_hpo_synthetic(delayed(evaluate_parameter_setting_synthetic)(parameter_setting) for parameter_setting in parameter_grid)

    sorted_evaluation_results_hpo_synthetic = sorted(evaluation_results_hpo_synthetic, key=lambda x: x[0], reverse=True)
    print(sorted_evaluation_results_hpo_synthetic[:5])

In [9]:
if False:
    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    sorted_evaluation_results_hpo_real_dict = {}

    for identifier in identifier_list:
        parallel_hpo_real = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
        evaluation_results_hpo_real = parallel_hpo_real(delayed(evaluate_parameter_setting_real_world)(parameter_setting, identifier) for parameter_setting in parameter_grid)

        sorted_evaluation_results_hpo_real = sorted(evaluation_results_hpo_real, key=lambda x: x[0], reverse=True)

        sorted_evaluation_results_hpo_real_dict[identifier] = sorted_evaluation_results_hpo_real

        print(sorted_evaluation_results_hpo_real[:5])
        